In [50]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil

In [51]:
def new_driver():
    try:
        shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
    except FileNotFoundError:
        pass

    subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동


    option = Options()
    option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0] #크롬 버전
    try:
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
        return driver
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
        return driver
    
    driver.implicitly_wait(10)
    
    

In [52]:
driver=new_driver()

In [54]:
# 사이트 주소는 나이키 남성신발 
path='https://www.nike.com/kr/ko_kr/w/men/fw'
driver.get(path) 
driver.implicitly_wait(5)  #5초전에 끝나면 탈출 time.sleep과 차이 

In [55]:
from selenium.webdriver import ActionChains


last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # id가 something 인 element 를 찾음
    #  = driver.find_element_by_class_name('btn-pagination more')
    some_tag=driver.find_element_by_xpath('/html/body/section/section/section/article/div/div[2]/div')
    
    # # somthing element 까지 스크롤
    action = ActionChains(driver)
    action.move_to_element(some_tag).perform()
    driver.execute_script("window.scrollTo(0, window.scrollY + 500);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [56]:
# 제품 url들 수집
url_list = []
title_list = []
subtitle_list= []
color_list=[]
price_list=[]

# URL_raw 크롤링 시작
products = ".a-product-image-link"
products_title=".item-title"
products_subtitle=".product-subtitle > .text-color-secondary"
products_color=" .product-colorways-number"
products_price=".product-display-price"

product_raw = driver.find_elements_by_css_selector(products)
product_title = driver.find_elements_by_css_selector(products_title)
product_subtitle = driver.find_elements_by_css_selector(products_subtitle)
product_color = driver.find_elements_by_css_selector(products_color)
product_price = driver.find_elements_by_css_selector(products_price)

# product_raw[0]
# product_name_raw[0]
# product_subtitle[0]

In [57]:
# 크롤링한 url 정제 시작
for product in product_raw:
    url = product.get_attribute('href')
    url_list.append(url)
# time.sleep(1)

#제목 크롤링
for product in product_title:
    title = product.text
    title_list.append(title)    

for product in product_subtitle:
     subtitle_list.append(product.text)

for product in product_color:
     color_list.append(product.text)
        
for product in product_price:
    price_list.append(product.text)            

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
print('subtitle갯수: ', len(subtitle_list))
print('color갯수: ', len(color_list))
print('price갯수: ', len(price_list))



url갯수:  644
title갯수:  644
subtitle갯수:  644
color갯수:  644
price갯수:  644


In [58]:
df = pd.DataFrame({'url':url_list, 
                   'title':title_list,
                   'subtitle':subtitle_list,
                   'color':color_list,
                   'price':price_list})

In [59]:
df

,url,title,subtitle,color,price
0,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...,조던 델타 2,남성 신발 라이프스타일,6 컬러,"159,000 원"
1,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...,조던 델타 2,남성 신발 라이프스타일,6 컬러,"159,000 원"
2,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...,나이키 메트콘 7 MF,성인공용 트레이닝화,1 컬러,"169,000 원"
3,https://www.nike.com/kr/ko_kr/t/women/fw/nike-...,나이키 에어맥스 퓨리오사 NRG,여성 신발 라이프스타일,1 컬러,"189,000 원"
4,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...,나이키 에어맥스 PRE-DAY LX,남성 신발 라이프스타일,2 컬러,"169,000 원"
...,...,...,...,...,...
639,https://www.nike.com/kr/ko_kr/t/men/fw/golf/CI...,나이키 에어맥스 1 G,남성 골프화,1 컬러,"159,000 원"
640,https://www.nike.com/kr/ko_kr/t/women/fw/nike-...,나이키 아수나,여성 슬라이드,2 컬러,"49,000 원"
641,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...,나이키 SB 셰인,성인공용 스케이트보드화,1 컬러,"79,200 원"
642,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...,나이키 케파 카이,남성 끈 샌들,2 컬러,"39,000 원"


In [60]:
uni_items=df.drop_duplicates(['title','color','subtitle','price'],keep='last') #중복 제거  

In [61]:
uni_items

,url,title,subtitle,color,price
2,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...,나이키 메트콘 7 MF,성인공용 트레이닝화,1 컬러,"169,000 원"
3,https://www.nike.com/kr/ko_kr/t/women/fw/nike-...,나이키 에어맥스 퓨리오사 NRG,여성 신발 라이프스타일,1 컬러,"189,000 원"
5,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...,나이키 에어맥스 PRE-DAY LX,남성 신발 라이프스타일,2 컬러,"169,000 원"
6,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...,나이키 에어맥스 PRE-DAY LX,남성 신발 라이프스타일,1 컬러,"169,000 원"
7,https://www.nike.com/kr/ko_kr/t/women/fw/nike-...,나이키 에어맥스 PRE-DAY LX,여성 신발 라이프스타일,1 컬러,"169,000 원"
...,...,...,...,...,...
639,https://www.nike.com/kr/ko_kr/t/men/fw/golf/CI...,나이키 에어맥스 1 G,남성 골프화,1 컬러,"159,000 원"
640,https://www.nike.com/kr/ko_kr/t/women/fw/nike-...,나이키 아수나,여성 슬라이드,2 컬러,"49,000 원"
641,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...,나이키 SB 셰인,성인공용 스케이트보드화,1 컬러,"79,200 원"
642,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...,나이키 케파 카이,남성 끈 샌들,2 컬러,"39,000 원"


In [ ]:
uni_itmes.to_parquet('nike.parquet')